In [1]:
!wget https://dl.google.com/linux/chrome/deb/pool/main/g/google-chrome-stable/google-chrome-stable_94.0.4606.71-1_amd64.deb
!dpkg -i google-chrome-stable_94.0.4606.71-1_amd64.deb
!apt-get -fy install

In [2]:
# install chromedriver and selenium
!pip install chromedriver-py==94.0.4606.41
!pip install selenium

In [3]:
# testing the installation
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from chromedriver_py import binary_path

In [4]:
!pip install tqdm

In [5]:
# creating a fuctnion to create a driver with given parameters
def add_driver_options(options):
    """
    This function sets driver options ( given as parameter)
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    This function create an instace of the web driver with the options below:
    """
    driver_config = {
        "executable_path": binary_path,
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
        ],
    }
    options = add_driver_options(driver_config["options"])
    driver = webdriver.Chrome(
        executable_path=driver_config["executable_path"], options=options
    )
    return driver

In [6]:
import time
from tqdm import tqdm
from selenium import webdriver

In [7]:
driver = initialize_driver()

In [8]:
def get_brands_cars_list(home_page_url):
    driver.get(home_page_url)
    brands = driver.find_elements(by=By.XPATH, value="//body/div[@id='brand']/div[1]/ul[1]/li/a[1]")
    brands_urls = [el.get_attribute('href') for el in brands]
    return brands_urls

In [9]:
def get_cars_urls(page):
    driver.get(page)
    cars = driver.find_elements(by=By.XPATH, value="//div[@id='search-container']/div[@id='result']/ul[@class='items']/li/a[1]")
    cars_urls = [el.get_attribute('href') for el in cars]
    return cars_urls

In [10]:
def get_next_page(url):
    driver.get(url)
    next_page = driver.find_elements(by=By.XPATH,value="//ul[@class='pages']//a")
    try:
        right_chevron = next_page[-1].find_element(by=By.XPATH, value="./i[@class='fa fa-chevron-right']")
        if right_chevron:
            return next_page[-1].get_attribute('href')
    except:
        return None

In [11]:
def get_car_data(car_url,driver):
    caracteristics={}
    
    try:
        driver.get(car_url)
        car_name=driver.find_element(by=By.XPATH,value="//body/div[@id='annonce']/div[1]/div[1]/h3[1]").text
        version_name=driver.find_element(by=By.XPATH,value="//body/div[@id='annonce']/div[1]/div[1]/h4[1]").text
        price = driver.find_element(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[1]/p[1]").text
        temp2={"Brand":car_name.split()[0],"Model":' '.join(car_name.split()[1:]), "Version":version_name,"Price":price}
        caracteristics.update(temp2)
        caracteristics_ = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/ul[1]/li")
        for element in caracteristics_:
            prop_name=element.find_element(by=By.XPATH,value=".//p[@class='titre']").text
            prop_value=element.find_element(by=By.XPATH,value=".//p[@class='tag']").text
            tmp={prop_name:prop_value}
            caracteristics.update(tmp)
        caracteristics_ = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/ul[1]")
        caracteristics_blocs = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[@class='cell']")
        caracteristics_blocs =caracteristics_blocs+ driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[@class='cell cell-93']")
        for i in range(len(caracteristics_blocs)):
            bloc=caracteristics_blocs[i]
            property_name="./ul[1]/li/p[@class='param']"
            property_value = "./ul[1]/li/p[@class='value']"
            names = bloc.find_elements(by=By.XPATH,value=property_name)
            values = bloc.find_elements(by=By.XPATH, value=property_value)
            param_values=[]
            for value in values:
                try:
                    image = value.find_element(by=By.XPATH, value="./img[1]")
                    if image.get_attribute("src")=="https://www.wandaloo.com/imgs/non.png":
                        param_values.append("non")
                    elif image.get_attribute("src")=="https://www.wandaloo.com/imgs/oui.png":
                        param_values.append("oui")
                except:
                    param_values.append(value.text)

            param_names = [el.text for el in names]
            temp = {param_names[i]: param_values[i] for i in range(len(param_names))}
            caracteristics.update(temp)
    except:
        print("an url has been skiped!")
    return caracteristics


In [12]:
def get_car_data2(car_url):
    caracteristics={}
    driver.get(car_url)
    car_name=driver.find_element(by=By.XPATH,value="//body/div[@id='annonce']/div[1]/div[1]/h3[1]").text
    version_name=driver.find_element(by=By.XPATH,value="//body/div[@id='annonce']/div[1]/div[1]/h4[1]").text
    price = driver.find_element(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[1]/p[1]").text
    temp2={"Brand":car_name.split()[0],"Model":' '.join(car_name.split()[1:]), "Version":version_name,"Price":price}
    caracteristics.update(temp2)
    caracteristics_ = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/ul[1]/li")
    for element in caracteristics_:
        prop_name=element.find_element(by=By.XPATH,value=".//p[@class='titre']").text
        prop_value=element.find_element(by=By.XPATH,value=".//p[@class='tag']").text
        tmp={prop_name:prop_value}
        caracteristics.update(tmp)
    caracteristics_ = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/ul[1]")
    caracteristics_blocs = driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[@class='cell']")
    caracteristics_blocs =caracteristics_blocs+ driver.find_elements(by=By.XPATH, value="//body/div[@id='annonce']/div[1]/div[@class='cell cell-93']")
    for i in range(len(caracteristics_blocs)):
        bloc=caracteristics_blocs[i]
        property_name="./ul[1]/li/p[@class='param']"
        property_value = "./ul[1]/li/p[@class='value']"
        names = bloc.find_elements(by=By.XPATH,value=property_name)
        values = bloc.find_elements(by=By.XPATH, value=property_value)
        param_values=[]
        for value in values:
            try:
                image = value.find_element(by=By.XPATH, value="./img[1]")
                if image.get_attribute("src")=="https://www.wandaloo.com/imgs/non.png":
                    param_values.append("non")
                elif image.get_attribute("src")=="https://www.wandaloo.com/imgs/oui.png":
                    param_values.append("oui")
            except:
                param_values.append(value.text)

        param_names = [el.text for el in names]
        temp = {param_names[i]: param_values[i] for i in range(len(param_names))}
        caracteristics.update(temp)
    return caracteristics

In [13]:
get_car_data2("https://www.wandaloo.com/occasion/alfa-romeo-giulietta-diesel-occasion-casablanca-maroc/33724.html")

In [14]:
def get_all_pages_urls(brand):
    urls_list=[]
    temp_url=brand
    Next=get_next_page(temp_url)
    while Next!=None:
        urls_list.append(temp_url)
        temp_url=Next
        Next=get_next_page(temp_url)
    urls_list.append(temp_url)
    return urls_list

In [15]:
get_cars_urls("https://www.wandaloo.com/occasion/?marque=1&modele=0=budget=0&categorie=0&moteur=0&transmission=0&vendeur=0&abonne=0&equipement=-&ville=0&mo=za&pg=1")

In [16]:
get_all_pages_urls('https://www.wandaloo.com/occasion/?marque=1&modele=0=budget=0&categorie=0&moteur=0&transmission=0&vendeur=0&abonne=0&equipement=-&ville=0&mo=za&pg=1')

In [19]:
def get_all_urls(verbose=True):
    pages_urls=[]
    urls=[]
    
    brands_urls=get_brands_cars_list("https://www.wandaloo.com/occasion/voiture-occasion-maroc-annonce.html")
    if verbose:
        print("============== {} Car brand has been found ==============".format(len(brands_urls)))
        print("============== Reading all pages, please wait... ==============")
    for i in tqdm(range(len(brands_urls))):
        brand_pages=get_all_pages_urls(brands_urls[i])
        pages_urls=pages_urls+brand_pages
    if verbose:
        print("============== Getting all annoucments, please wait... ==============")
    for i in tqdm(range(len(pages_urls))):
        cars=get_cars_urls(pages_urls[i])
        urls=urls+cars
    if verbose:
        print("============== All data urls has been fetched, total of {} car ==============".format(len(urls)))
        print("============== Getting Data =================")
        print("=========== urls: =============\n")
        print(urls)
    return urls




In [18]:
def split(List, n_batches):
    k, m = divmod(len(List), n_batches)
    return list(List[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

In [20]:
def get_batch_data(batch,start_index,frames,i):
    batch_data=pd.DataFrame()
    temp_driver= initialize_driver()
    for j in tqdm(range(len(batch)),colour="blue"):
        data=get_car_data(batch[j],temp_driver)
        df_temp = pd.DataFrame(data, index=[str(j+start_index)])
        batch_data = pd.concat([batch_data, df_temp])
    frames[i]=batch_data

In [ ]:
import time
from threading import Thread
def main(n_batches=30,verbose=True):
    urls=get_all_urls()
    df=pd.DataFrame()
    threads=[None]*n_batches
    frames=[None]*n_batches
    batches=split(urls,n_batches)
    start=0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index=0
        t = Thread(target=get_batch_data, args=(batches[i],start_index,frames,i,))
        threads[i]=t
        t.start()
    for t in threads:
        t.join()
    if verbose:
        print("============== Data has been scraped successfully, check your output directory ! ==============")
    df=pd.concat(frames)
    df.to_csv('old_cars_data_final.csv')
    return df
old_cars=pd.DataFrame()
if __name__ == "__main__":
    old_cars=main()
    old_cars.to_csv('old_cars_data_final.csv')

In [ ]:
data=pd.read_csv("./old_cars_data1.csv")

In [ ]:
data.head()

In [ ]:
df=pd.read_csv("./new.csv")

In [ ]:
df.drop_duplicates()